In [1]:
%matplotlib inline
import sys
import os
sys.path.append("/Users/chernals/cernbox/Work/packages")
#sys.path.append("/Users/chernals/cernbox/Work/packages/georges")
import matplotlib.pyplot as plt
import georges
PATH = "/Users/chernals/Dropbox/IBA/Work/IBA-Optics/beamlines"

In [19]:
bl_track.line[['BEAM']].iloc[0].values[0].mean

X      8.997435e-03
PX     8.172793e-04
Y     -3.417715e-03
PY     7.068647e-04
DPP    2.220446e-16
dtype: float64

In [2]:
brho = georges.physics.momentum_to_brho(georges.physics.energy_to_momentum(177.5))
context = {
        'PARTICLE': 'PROTON',
        'PC': georges.physics.energy_to_momentum(177.5),
        'BETAX'   : 0.0846155,
        'BETAY'   : 0.0846155,
        'ALPHAX': 0.0,
        'ALPHAY': 0.0,
        'DELTAP': 0.0,
        'DPP': 0.5e-2,
        'IQ1E': 2.741/10/brho/0.0325*0.297/0.295,
        'IQ2E': -5.745/10/brho/0.0325*0.297/0.295,
        'IQ3E': 4.079/10/brho/0.0325*0.297/0.295,
        'IQ47E': -3.062/10/brho/0.0325,
        'IQ56E': 2.66/10/brho/0.0325,
        'IQ8E': 3.5484/10/brho/0.0325,
        'IQ9E': -4.0276/10/brho/0.0325,
        'IQ10E': 4.0352/10/brho/0.0325,
        'MOMENTUM_SLITS_OPENING' : 1.015,
        'DIVERGENCE_SLITS_OPENING_X' : 1.015,
        'DIVERGENCE_SLITS_OPENING_Y' : 1.015,
        'N_TRACKING': 5000,
        'EMITX': 14.3e-6,
        'EMITY': 14.3e-6,
    }
m = georges.madx.Madx(madx='/usr/local/bin/madx', path=PATH, context=context)
bl = georges.Beamline('ess', path=PATH, prefix='generic', elements='elements')
#bl_twiss= georges.madx.twiss(line=bl, madx=m)
#b = georges.Beam(energy=200).from_5d_multigaussian_distribution(5,
#                                                   XRMS=0.01,
#                                                   PXRMS=0.001,
#                                                   YRMS=0.01,
#                                                   PYRMS=0.001,
#                                                   DPPRMS=0.00)
#bl_track = georges.madx.track(line=bl, madx=m, beam=b, ptc=False)
#from georges.plotting import *
#with plt.style.context('word'):
#    fig = plt.figure(1)

#    ax1 = fig.add_subplot(111)
#    prepare(ax1, bl)
#    aperture(ax1, bl)
    #twiss(ax1, bl_twiss)
    #track(ax1, bl, b)


In [5]:
l = georges.madx.twiss(line=bl, madx=m)

In [21]:
l.line

,KEYWORD,S,BETX,ALFX,MUX,BETY,ALFY,MUY,X_TWISS,PX,...,E2,HGAP,THICK,PLUG,CHAMBER,AT_CENTER,AT_ENTRY,AT_EXIT,ORBIT_LENGTH,PHYSICAL
NAME,,,,,,,,,,,,,,,,,,,,,
ESS$START,MARKER,0.000000,0.084615,0.000000,0.000000,0.084615,0.000000,0.000000,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,True
P1E,INSTRUMENT,0.000000,0.084615,-0.000000,0.000000,0.084615,-0.000000,0.000000,0,0,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,True
DRIFT_0,DRIFT,0.133089,0.293946,-1.572865,0.159868,0.293946,-1.572865,0.159868,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,True
Q1E,QUADRUPOLE,0.413678,1.961565,-3.685770,0.218238,2.259975,-6.201367,0.217557,0,0,...,NaN,NaN,NaN,K1,0.005,0.413678,0.266178,0.561178,0.295000,True
DRIFT_1,DRIFT,0.727676,4.025219,-3.447369,0.235501,9.454068,-16.298389,0.228720,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,True
Q2E,QUADRUPOLE,1.041674,7.744754,-13.917553,0.245057,18.845015,0.823088,0.232221,0,0,...,NaN,NaN,NaN,K1,0.005,1.041674,0.894174,1.189174,0.295000,True
DRIFT_2,DRIFT,1.258175,19.839498,-41.154385,0.247984,12.347171,19.841389,0.234363,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,True
Q3E,QUADRUPOLE,1.474675,36.970759,-24.396182,0.249210,6.193905,7.639959,0.238452,0,0,...,NaN,NaN,NaN,K1,0.005,1.474675,1.327175,1.622175,0.295000,True
DRIFT_3,DRIFT,1.918006,32.820611,10.059326,0.251129,3.734932,1.700358,0.253934,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,True


ValueError: cannot reindex from a duplicate axis

In [2]:
b = Beam(energy=200).from_5d_multigaussian_distribution(20,
                                               XRMS=0.01,
                                               PXRMS=0.001,
                                               YRMS=0.01,
                                               PYRMS=0.001,
                                               DPPRMS=0.00)

context = {
    'PARTICLE': 'PROTON',
    #'PC': energy_to_momentum(177.5),
    'BETAX'   : 0.0846155,
    'BETAY'   : 0.0846155,
    'ALPHAX': 0.0,
    'ALPHAY': 0.0,
    'DELTAP': 0.0,
    'DPP': 0.5e-2,
    'IQ1G': 2.741, # K1 IQ1G
    'MOMENTUM_SLITS_OPENING' : 0.015,
    'DIVERGENCE_SLITS_OPENING_X' : 0.015,
    'DIVERGENCE_SLITS_OPENING_Y' : 0.015,
    'N_TRACKING': 5000,
    'EMITX': 14.3e-6,
    'EMITY': 14.3e-6,
    'BETAREL': 0.5,
}

bl = Beamline('cgtr', beam=b, elements='beamlines/elements', path=PATH, prefix='beamlines/generic')
# If using a direct path
#bl = Beamline('cgtr', beam=b, elements='elements', path='path/to/files')
